In [8]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import matplotlib.pyplot as plt
from time import *
import numpy as np

In [2]:
class Inception(tf.keras.layers.Layer):
    # 设置模块的构成
    def __init__(self, c1, c2, c3, c4):
        super().__init__()
        # 线路1:1*1 RELU same c1
        self.p1_1 = tf.keras.layers.Conv2D(
            c1, kernel_size=1, activation="relu", padding="same")
        # 线路2:1*1 RELU same c2[0]
        self.p2_1 = tf.keras.layers.Conv2D(
            c2[0], kernel_size=1, activation="relu", padding="same")
        # 线路2:3*3 RELU same c2[1]
        self.p2_2 = tf.keras.layers.Conv2D(
            c2[1], kernel_size=3, activation="relu", padding='same')
        # 线路3:1*1 RELU same c3[0]
        self.p3_1 = tf.keras.layers.Conv2D(
            c3[0], kernel_size=1, activation="relu", padding="same")
        # 线路3:5*5 RELU same c3[1]
        self.p3_2 = tf.keras.layers.Conv2D(
            c3[1], kernel_size=5, activation="relu", padding='same')
        # 线路4: max-pool
        self.p4_1 = tf.keras.layers.MaxPool2D(
            pool_size=3, padding="same", strides=1)
        # 线路4:1*1
        self.p4_2 = tf.keras.layers.Conv2D(
            c4, kernel_size=1, activation="relu", padding="same")

    # 前行传播过程
    def call(self, x):
        # 线路1
        p1 = self.p1_1(x)
        # 线路2
        p2 = self.p2_2(self.p2_1(x))
        # 线路3
        p3 = self.p3_2(self.p3_1(x))
        # 线路4
        p4 = self.p4_2(self.p4_1(x))
        # concat
        outputs = tf.concat([p1, p2, p3, p4], axis=-1)
        return outputs

In [3]:
# 数据集加载函数，指明数据集的位置并统一处理为imgheight*imgwidth的大小，同时设置batch
def load_data(dir_path):
    fopen = open(dir_path, 'r')
    lines = fopen.read().splitlines()   # 逐行读取txt
    count = len(open(dir_path, 'r').readlines())      # 计算txt有多少行
 
    data_set = np.empty((count, 27, 1024, 4), dtype="float32")
    label = np.zeros((count), dtype="uint8")
 
    i = 0
    for line in lines:
 
        line = line.split(" ")          # 利用空格进行分割
 
        # img = Image.open(line[0])
        sample = np.load(line[0])
        # print(i, sample.size)
        # img = skimage.io.image(line[0])
        label[i] = int(line[1])
 
        # img = img.convert('L')          # 转灰度图像
        array = np.asarray(sample, dtype="float32")
        data_set[i, :, :, :] = array
 
        i += 1
 
    return data_set, label

In [4]:
# 构建mobilenet模型
# 模型加载，指定图片处理的大小和是否进行迁移学习
def model_load(IMG_SHAPE=(27, 1024, 4), class_num=10):
    # 微调的过程中不需要进行归一化的处理
    # 加载预训练的mobilenet模型
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')
    # 将模型的主干参数进行冻结
    base_model.trainable = False


    model = tf.keras.models.Sequential([
        # 进行归一化的处理
        tf.keras.layers.experimental.preprocessing.Rescaling(1. / 127.5, offset=-1, input_shape=IMG_SHAPE),
        # 设置主干模型
        base_model,
        # 对主干模型的输出进行全局平均池化
        tf.keras.layers.GlobalAveragePooling2D(),
        # 通过全连接层映射到最后的分类数目上
        tf.keras.layers.Dense(class_num, activation='softmax')
    ])
    model.summary()
    # 模型训练的优化器为adam优化器，模型的损失函数为交叉熵损失函数
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [5]:
# 展示训练过程的曲线
def show_loss_acc(history):
    # 从history中提取模型训练集和验证集准确率信息和误差信息
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # 按照上下结构将图画输出
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.savefig('results/results_mobilenet.png', dpi=100)

In [6]:
def train(epochs):
    # 开始训练，记录开始时间
    begin_time = time()
    # todo 加载数据集， 修改为你的数据集的路径
#     train_ds, val_ds, class_names = data_load("H:/CNN2/new_data/train",
#                                            "H:/CNN2/new_data/val", 224, 224, 16)
    train_list = './slice_data/train_label.txt'
    val_list = './slice_data/val_label.txt'
    x_sample, x_label = load_data(train_list)
    y_sample, y_label = load_data(val_list)
    x_label = tf.squeeze(x_label)
    x_label = tf.one_hot(x_label, depth = 10)
    y_label = tf.squeeze(y_label)
    y_label = tf.one_hot(y_label, depth = 10)
    db_train = tf.data.Dataset.from_tensor_slices((x_sample,x_label)).batch(256)
    db_val = tf.data.Dataset.from_tensor_slices((y_sample,y_label)).batch(256)
#     print(class_names)
    # 加载模型
    model = model_load()
    # 指明训练的轮数epoch，开始训练
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    # todo 保存模型， 修改为你要保存的模型的名称
    model.save("models/mobilenet_picture_video_text.h5")
    # 记录结束时间
    end_time = time()
    run_time = end_time - begin_time
    print('该循环程序运行时间：', run_time, "s")  # 该循环程序运行时间： 1.4201874732
    # 绘制模型训练过程图
    show_loss_acc(history)

In [9]:
if __name__ == '__main__':
    train(epochs=30)

2022-04-11 14:41:51.555035: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: The input must have 3 channels; Received `input_shape=(27, 1024, 4)`